#  **Data Wrangling**

### Introduction
For this project a personality scores csv file is imported as a dataframe, its data is cleaned and its rows(which represent each individual) are used to create new columns that contain the total values subscales. The dataframe is then merged with a csv file (department) which was also cleaned before the merge to create a new dataframe, which consists of personality scores and departments. From the data contained in the merged dataframe a new dataframe is created, which provides a risk scale count of low-risk or high-risk individals within each departments.

The following concepts will be used to show an understanding of some data wrangling techniques

       * Merging dataframes
       * Filtering dataframes
       * Manipulating rows and columns

Import required modules

In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval

Import dataset, personality_scores.csv.

In [2]:
df = pd.read_csv("../data/personality_scores.csv", sep=";")

### Cleaning data

Remove any duplicates from dataframe based on "ID".

In [3]:
personality_scores_df = df.drop_duplicates(subset="ID", keep=False)
personality_scores_df.shape

(1555, 70)

Assert that the new dataframe(personality_scores_df) is the length of the unique entries of the original dataframe(df).

In [4]:
assert df.nunique(axis=1).size == personality_scores_df.shape[0]

Remove "Section 5 of 6" from column name to give clearier column names.

In [5]:
personality_scores_df.rename(lambda name: name.replace("Section 5 of 6 [", ""), axis=1, inplace=True)
personality_scores_df.rename(lambda name: name.replace(".]", ""), axis=1, inplace=True)
personality_scores_df.rename(lambda name: name.replace("]", ""), axis=1, inplace=True)
personality_scores_df.head()

,ID,I am always prepared,I am easily disturbed,I am exacting (demanding) in my work,I am full of ideas,I am interested in people,I am not interested in abstract ideas,I am not interested in other people's problems,I am not really interested in others,I am quick to understand things,...,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,IPIP_HIGH_RISK
0,0,"(3, 5)","(4, 5)","(3, 5)","(5, 5)","(2, 3)","(5, 3)","(2, 3)","(2, 5)","(5, 5)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"(3, 5)","(4, 5)","(3, 5)","(5, 5)","(2, 5)","(5, 3)","(2, 5)","(2, 5)","(5, 5)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,"(3, 5)","(4, 3)","(3, 3)","(5, 5)","(2, 5)","(5, 5)","(2, 5)","(2, 5)","(5, 5)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,"(3, 5)","(4, 5)","(3, 3)","(5, 5)","(2, 5)","(5, 3)","(2, 3)","(2, 3)","(5, 3)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,"(3, 3)","(4, 5)","(3, 3)","(5, 3)","(2, 3)","(5, 3)","(2, 3)","(2, 3)","(5, 5)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Columns with value = NaN are removed.

In [6]:
for column in personality_scores_df.columns:
    if "Unnamed" in column:
        del personality_scores_df[column]
    if "IPIP_HIGH_RISK" in column:
         del personality_scores_df[column]
            
personality_scores_df.shape

(1555, 51)

### Manipulating rows and columns

Create new columns containing the total score of each of the personality test subscales by making use of a while loop that will iterate through each row. The keys of the dictionary are renamed to represent the column names in the dataframe.


In [7]:
def add_total_score_columns():
    counter = 0
    list_of_scores_dict = []
    while counter < personality_scores_df.shape[0]:
        scores_list = personality_scores_df.iloc[counter].to_list()
        scores_list.pop(0)
        scores_dict = {1:0, 2:0, 3:0, 4:0, 5:0}
        for score in scores_list:
            if score != 0:
                score = literal_eval(score)
                scores_dict[score[0]] = scores_dict.get(score[0])
                scores_dict[score[0]] += score[1]
            else:
                continue
        scores_dict["Extraversion"] = scores_dict.pop(1)
        scores_dict["Agreeableness"] = scores_dict.pop(2)
        scores_dict["Conscientiousness"] = scores_dict.pop(3)
        scores_dict["Emotional Stability/Neuroticism"] = scores_dict.pop(4)
        scores_dict["Intellect/Imaginations"] = scores_dict.pop(5)
        list_of_scores_dict.append(scores_dict)
        counter +=1
    total_scores_df = pd.DataFrame(list_of_scores_dict)
    total_scores_df = total_scores_df.astype({"Extraversion":int, "Agreeableness":int, "Conscientiousness":int, "Emotional Stability/Neuroticism":int, "Intellect/Imaginations":int})
    return total_scores_df

    

The personality_scores and total_scores dataframes are then joined along the matching rows.

In [8]:
joining_dataframe = add_total_score_columns()
personality_scores = pd.concat([personality_scores_df, joining_dataframe], axis=1)
personality_scores.head()

,ID,I am always prepared,I am easily disturbed,I am exacting (demanding) in my work,I am full of ideas,I am interested in people,I am not interested in abstract ideas,I am not interested in other people's problems,I am not really interested in others,I am quick to understand things,...,I sympathize with others' feelings,I take time out for others,I talk to a lot of different people at parties,I use difficult words,I worry about things,Extraversion,Agreeableness,Conscientiousness,Emotional Stability/Neuroticism,Intellect/Imaginations
0,0,"(3, 5)","(4, 5)","(3, 5)","(5, 5)","(2, 3)","(5, 3)","(2, 3)","(2, 5)","(5, 5)",...,"(2, 5)","(2, 5)","(1, 3)","(5, 1)","(4, 3)",30,40,48,36,42
1,1,"(3, 5)","(4, 5)","(3, 5)","(5, 5)","(2, 5)","(5, 3)","(2, 5)","(2, 5)","(5, 5)",...,"(2, 5)","(2, 5)","(1, 5)","(5, 3)","(4, 3)",42,46,46,40,42
2,2,"(3, 5)","(4, 3)","(3, 3)","(5, 5)","(2, 5)","(5, 5)","(2, 5)","(2, 5)","(5, 5)",...,"(2, 5)","(2, 5)","(1, 3)","(5, 1)","(4, 3)",28,40,40,38,42
3,3,"(3, 5)","(4, 5)","(3, 3)","(5, 5)","(2, 5)","(5, 3)","(2, 3)","(2, 3)","(5, 3)",...,"(2, 5)","(2, 5)","(1, 5)","(5, 1)","(4, 1)",30,38,38,40,38
4,4,"(3, 3)","(4, 5)","(3, 3)","(5, 3)","(2, 3)","(5, 3)","(2, 3)","(2, 3)","(5, 5)",...,"(2, 3)","(2, 5)","(1, 3)","(5, 1)","(4, 3)",28,34,46,38,36


Import dataset, departments.csv.

In [9]:
departments_df = pd.read_csv("../data/departments.csv", sep=";", index_col=0)

Merge departments and personality_scores dataframes

In [10]:
personality_and_departments_df = pd.merge(personality_scores, departments_df, how="outer", on="ID")
personality_and_departments_df.set_index("ID", inplace=True)
personality_and_departments_df.head()

,I am always prepared,I am easily disturbed,I am exacting (demanding) in my work,I am full of ideas,I am interested in people,I am not interested in abstract ideas,I am not interested in other people's problems,I am not really interested in others,I am quick to understand things,I am quiet around strangers,...,I take time out for others,I talk to a lot of different people at parties,I use difficult words,I worry about things,Extraversion,Agreeableness,Conscientiousness,Emotional Stability/Neuroticism,Intellect/Imaginations,Department
ID,,,,,,,,,,,,,,,,,,,,,
0,"(3, 5)","(4, 5)","(3, 5)","(5, 5)","(2, 3)","(5, 3)","(2, 3)","(2, 5)","(5, 5)","(1, 3)",...,"(2, 5)","(1, 3)","(5, 1)","(4, 3)",30,40,48,36,42,Data
1,"(3, 5)","(4, 5)","(3, 5)","(5, 5)","(2, 5)","(5, 3)","(2, 5)","(2, 5)","(5, 5)","(1, 3)",...,"(2, 5)","(1, 5)","(5, 3)","(4, 3)",42,46,46,40,42,Data
2,"(3, 5)","(4, 3)","(3, 3)","(5, 5)","(2, 5)","(5, 5)","(2, 5)","(2, 5)","(5, 5)","(1, 1)",...,"(2, 5)","(1, 3)","(5, 1)","(4, 3)",28,40,40,38,42,Data
3,"(3, 5)","(4, 5)","(3, 3)","(5, 5)","(2, 5)","(5, 3)","(2, 3)","(2, 3)","(5, 3)","(1, 3)",...,"(2, 5)","(1, 5)","(5, 1)","(4, 1)",30,38,38,40,38,Data
4,"(3, 3)","(4, 5)","(3, 3)","(5, 3)","(2, 3)","(5, 3)","(2, 3)","(2, 3)","(5, 5)","(1, 1)",...,"(2, 5)","(1, 3)","(5, 1)","(4, 3)",28,34,46,38,36,Data


Assert that the number of rows in the merged dataframe = number of rows in the departments dataframe.

In [11]:
assert len(personality_and_departments_df.index) == len(departments_df.index)


### Filtering dataframe

Assign applicants who scored less than 30 on Emotional stability, Conscientiousness and Agreeableness “high_risk” tag and the other applicants "low_risk" tag.

In [12]:
personality_and_departments_df["Risk"] = np.where((personality_and_departments_df["Agreeableness"] < 30) & (personality_and_departments_df["Conscientiousness"] < 30) & (personality_and_departments_df["Emotional Stability/Neuroticism"] < 30), "High risk", "Low risk") 

Print applicants who scored less than 30 on Emotional stability, Conscientiousness and Agreeableness.

In [13]:
personality_and_departments_df.loc[personality_and_departments_df["Risk"] == "High risk", ["Department"]]

,Department
ID,
881,Data
1197,Copywriting


Adjust "department" column values so that they are consistent throughout the column and create a new data frame with a count of the number of low and high-risk applicants within each department.

In [14]:
personality_and_departments_df.replace("Web dev", "Web Dev", inplace=True) 
risk_df = pd.DataFrame(personality_and_departments_df[["Risk", "Department"]])
risk_df = (risk_df.groupby(["Risk", "Department"]).size()).unstack(fill_value=0)
risk_df

Department,Copywriting,Data,Design,Strategy,Web Dev
Risk,,,,,
High risk,1,1,0,0,0
Low risk,325,328,120,449,331


### Conclusion
The data wrangling concepts: merging, filtering and manipulating rows and columns were used and the output data is as expected and has been verified. The notebook now contains two dataframes: The personality_and_departments, which contains total scores for subscales alongside the department and risk, which contains a count of low and high risk individuals for each department.